In [0]:
ddf = spark.read.format("parquet").options(header='true').load('/FileStore/tables/processedFiles/2020/yellow_tripdata.parquet')
display(ddf)

#creating tempview for analysis
ddf.createOrReplaceTempView("yellowTaxidata")

In [0]:
'''
Assumptions: 1) Negative tip amout and total amounts are converted to their positive values, since tip can't be negative.
             2) Considering that the data is from a perticular quarter, dates from forward months are resticted.
'''
tip_percent_quarter = spark.sql("SELECT loc_id	,quarter	,TipPercentage	,tip_amount FROM (	SELECT DOLocationID AS loc_id		,quarter		,tip_amount		,TipPercentage	,dense_rank() OVER (PARTITION BY quarter ORDER BY TipPercentage DESC ) AS max_tip_percent	FROM (		SELECT tpep_pickup_datetime AS pickup_datetime	,tpep_dropoff_datetime AS dropoff_datetime			,hours_diff	,trip_distance	,DOLocationID	,total_amount	,abs(tip_amount) AS tip_amount,quarter	,CASE WHEN abs(tip_amount) < (abs(total_amount) - abs(tip_amount)) THEN (ROUND((abs(tip_amount)) / (abs(total_amount) - abs(tip_amount)) * 100, 3))	ELSE 0	END AS TipPercentage FROM yellowTaxidata WHERE Quarter IN (1	,2)) a) b WHERE b.max_tip_percent = 1")
display(tip_percent_quarter)
        

loc_id,quarter,TipPercentage,tip_amount
239,1,99.773,8.78
152,2,99.802,25.25


In [0]:
'''
Assumption : 1) minimum hour difference between pickup and drop location should be 0.24
               2) Since data has a lot of outliers I needed to cap the speed between 10mph and 55 mph (Maximum permissible limit)
'''
speed_per_hour = spark.sql("select speed,hour from (SELECT speed ,hour,dense_rank() over (partition by hour order by speed desc) as speed_rank FROM ( SELECT tpep_pickup_datetime AS pickup_datetime,tpep_dropoff_datetime AS dropoff_datetime,extract(HOUR from tpep_pickup_datetime) as hour			,hours_diff	,abs(trip_distance) as trip_distance,DOLocationID,total_amount,tip_amount ,quarter ,ROUND((abs(trip_distance) / hours_diff),2) AS speed		FROM yellowTaxidata where Quarter < 2 and hours_diff >0.24        )a where  speed between 10 and 55   )b where speed_rank =1 order by hour")
display(speed_per_hour)

speed,hour
54.97,0
54.71,1
54.78,2
54.88,3
54.87,4
55,5
55,5
55,6
55,6
55,7


In [0]:
#creating dataframe to store values in SQL SERVER tables
fin_df_speed = spark.sql("SELECT hour as calendarHour,1 as calendarDay, 1 as calendarMonth, 2020 as calendarYear, Speed as maxSpeed FROM (	SELECT speed		,hour		,dense_rank() OVER (			PARTITION BY hour ORDER BY speed DESC			) AS speed_rank	FROM (		SELECT              tpep_pickup_datetime AS pickup_datetime			,tpep_dropoff_datetime AS dropoff_datetime			,extract(HOUR FROM tpep_pickup_datetime) AS hour			,hours_diff			,abs(trip_distance) AS trip_distance			,DOLocationID			,total_amount			,tip_amount			,quarter			,ROUND((abs(trip_distance) / hours_diff), 3) AS speed       FROM yellowTaxidata		WHERE Quarter < 2			AND hours_diff > 0.24 and  extract(day FROM tpep_pickup_datetime)= 1	) a	WHERE speed BETWEEN 10	AND 55	) b WHERE speed_rank = 1 ORDER BY hour")
fin_df_tip = spark.sql("SELECT quarter	,tip_amount ,2020 as year FROM (	SELECT DOLocationID AS loc_id		,quarter		,tip_amount		,TipPercentage	,dense_rank() OVER (PARTITION BY quarter ORDER BY TipPercentage DESC ) AS max_tip_percent	FROM (		SELECT tpep_pickup_datetime AS pickup_datetime	,tpep_dropoff_datetime AS dropoff_datetime			,hours_diff	,trip_distance	,DOLocationID	,total_amount	,abs(tip_amount) AS tip_amount,quarter	,CASE WHEN abs(tip_amount) < (abs(total_amount) - abs(tip_amount)) THEN (ROUND((abs(tip_amount)) / (abs(total_amount) - abs(tip_amount)) * 100, 3))	ELSE 0	END AS TipPercentage FROM yellowTaxidata WHERE Quarter IN (1	,2)) a) b WHERE b.max_tip_percent =1")

jdbcUrl = "jdbc:sqlserver://mysqlserver057.database.windows.net:1433;database=mysqldb"
connectionProperties = {
  "user" : "ashish",
  "password" : "Lenovo3000!",
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

try:
  fin_df_tip.write.jdbc(url=jdbcUrl, table="dbo.tip",mode = "overwrite", properties=connectionProperties )
except ValueError as error :
  print("Connector write failed", error)
    
try:
  fin_df_speed.write.jdbc(url=jdbcUrl, table="dbo.tripSpeed",mode = "overwrite", properties=connectionProperties )
except ValueError as error :
  print("Connector write failed", error)
